# Imports

In [1]:
import pickle
from typing import Literal
import numpy as np

# Load data

In [2]:
def load_dataset(path: str, prefix: Literal['train', 'test']='train'):
    ds = pickle.load(open(path, 'rb'))
    data = ds[f'{prefix}_data']
    labels = ds[f'{prefix}_labels']
    feature_names = ds['feature_names']
    return data, labels, feature_names

In [3]:
train_data, train_labels, train_feature_names = load_dataset('data_with_tiff/train_ds.bin')
train_img_data, train_img_labels, train_img_feature_names = load_dataset('data_with_tiff/train_imaging_ds.bin')
test_data, test_labels, test_feature_names = load_dataset('data_with_tiff/test_ds.bin', 'test')
test_img_data, test_img_labels, test_img_feature_names = load_dataset('data_with_tiff/test_imaging_ds.bin', 'test')

In [4]:
# def load_dataset(path: str, prefix: Literal['train', 'test']='train'):
#     ds = pickle.load(open(path, 'rb'))
#     data = ds[f'{prefix}_data']
#     labels = ds[f'{prefix}_labels']
#     return data, labels

# train_data, train_labels = load_dataset('data_with_tiff/train_ds.bin')
# train_img_data, train_img_labels = load_dataset('data_with_tiff/train_imaging_ds.bin')
# test_data, test_labels = load_dataset('data_with_tiff/test_ds.bin', 'test')
# test_img_data, test_img_labels = load_dataset('data_with_tiff/test_imaging_ds.bin', 'test')

In [5]:
# this feature contains NaNs
def remove_broken_feature(data, col_index=437):
    return np.delete(data, col_index, axis=1)

In [6]:
train_data = remove_broken_feature(train_data)
test_data = remove_broken_feature(test_data)

# Inspect data

In [7]:
train_data.shape, train_labels.shape, len(train_feature_names)

((362254, 448), (362254,), 449)

In [8]:
train_img_data.shape, train_img_labels.shape, len(train_img_feature_names)

((362254, 37), (362254,), 37)

In [9]:
test_data.shape, test_labels.shape, len(test_feature_names)

((63928, 448), (63928,), 449)

In [10]:
test_img_data.shape, test_img_labels.shape, len(test_img_feature_names)

((63928, 37), (63928,), 37)

In [11]:
type(train_data)

numpy.ndarray

In [12]:
train_data.dtype

dtype('float64')

In [13]:
train_data[0]

array([ 9.96600000e+03,  4.60000000e+01,  2.16000000e+02,  6.79870000e+04,
        1.40000000e+02,  4.85000000e+02,  9.02330000e+04,  1.05000000e+02,
        8.59000000e+02,  2.24270000e+04,  5.00000000e+01,  4.48000000e+02,
        1.39414700e+06,  7.66000000e+02,  1.82000000e+03,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  3.47000000e+02,  7.23000000e+02,  1.50000000e+02,
        1.65000000e+02,  1.45000000e+02,  1.59000000e+02,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  

In [14]:
np.isnan(train_data).any()

False

In [15]:
np.where(np.isnan(test_img_data).sum(axis=0))

(array([], dtype=int64),)

In [16]:
train_labels, train_labels.min(), train_labels.max(), train_labels.sum(), train_labels.shape, train_labels.dtype

(array([1, 0, 1, ..., 1, 0, 1]), 0, 1, 107110, (362254,), dtype('int64'))

# Baseline model
First I use the labels to filter out the debris in the train data. My idea is then to calculate the mean and std for each feature and to use this to find a range for each feature. For this I introduce 2 hyperparameters. The first one is `p`, describing the fraction of data points that should be in the range. The smaller this value is the stricter the gating is. For now I always take the center around the mean. The second hyperparameter I call `k`, describing the number of features actually used for the gating strategy. For this I will rank the features based on their descriptivness. For now I use the std as a proxy for this, with smaller std being interpreted as more informative. For this the features need to be normalized. For now I use a simple min max normalization per feature.

In [17]:
def filter_pure_sample_without_debris(feature_table, labels, label_to_use=0):
    assert feature_table.shape[0] == labels.shape[0]
    pure_sample = feature_table[labels == label_to_use]
    pure_samples_without_debris = pure_sample[pure_sample[:, -1] == 1]
    return pure_samples_without_debris[:, :-1]  # remove the last column (label if there is tiff)

In [18]:
pure_sample_without_debris = filter_pure_sample_without_debris(train_data, train_labels)
pure_sample_without_debris.shape

(2527, 447)

## Trying to setup ranking
See text below.

In [19]:
# np.isnan(pure_sample_without_debris).any()

In [20]:
# mins = pure_sample_without_debris.min(axis=0)
# maxs = pure_sample_without_debris.max(axis=0)
# np.isnan(mins).any(), np.isnan(maxs).any()

In [21]:
# max_min_diff = maxs - mins
# np.where(max_min_diff == 0)

In [22]:
# for i in [ 94,  95,  96, 427, 428, 429, 430, 431, 432, 433, 436, 438, 440,
#         441, 445, 446]:
#     print(pure_sample_without_debris[:, i].sum())

In [23]:
# pure_sample_without_debris[:, 427]

In [24]:
# norms = (pure_sample_without_debris - mins) / (maxs - mins)
# np.isnan(norms).any()

The problem here is that there are features that have constant value for all events. This screws up normalization. Because it is dividing by zero at some point. Also it is unclear if these features are informative at all. If not they should be rmeoved. Here I would just remove them to be able to normalize and thus to be able to produce a ranking based on std. Since it is unclear if the ranking actually helps, I just leave it as is right now and continue without a ranking, just using a range for each feature.

In [25]:
# def normalize_each_feature_seperately(data):
#     data = data.astype('float128')
#     print(data.dtype)
#     data -= data.min(axis=0)
#     data /= data.max(axis=0)
#     return data

In [26]:
# np.isnan(pure_sample_without_debris).any()

In [27]:
# normalized_pure_sample_without_debris = normalize_each_feature_seperately(pure_sample_without_debris)
# np.isnan(normalized_pure_sample_without_debris).any()

In [28]:
# def different_norm(data):
#     minimum, maximum = data.min(axis=0), data.max(axis=0)
#     return (data - minimum) / (maximum - minimum)

In [29]:
# normalized_pure_sample_without_debris = different_norm(pure_sample_without_debris)
# np.isnan(normalized_pure_sample_without_debris).any()

In [30]:
# def create_ranking_based_on_std(feature_table):
#     std = feature_table.std(axis=0)
#     print(std)
#     ranked_indices = np.argsort(std)
#     print(ranked_indices)
#     print(std[ranked_indices])
#     return ranked_indices

In [31]:
# ranked_features = create_ranking_based_on_std(pure_sample_without_debris)

In [32]:
# pure_sample_without_debris[:, -1].min(), pure_sample_without_debris[:, -1].max(), pure_sample_without_debris[:, -1].sum()

## Setup a range for each feature
no ranking for now, see text above

In [33]:
def get_bounds_for_each_feature(feature_table, p):
    assert 0 <= p <= 1
    lower_quantile = (1-p) / 2
    upper_quantile = 1 - lower_quantile
    bounds = np.zeros((feature_table.shape[1], 2))
    bounds[:, 0] = np.quantile(feature_table, lower_quantile, axis=0)
    bounds[:, 1] = np.quantile(feature_table, upper_quantile, axis=0)
    return bounds

# one could also think about using p rather as measure of how far we
    # go from the min/max value of the feature instead of interpreting it
    # as a sort of percentile
    # somehting like this:
    # mins = feature_table.min(axis=0)
    # maxs = feature_table.max(axis=0)
    # diffs = maxs - mins
    # bounds = np.zeros((feature_table.shape[1], 2))
    # bounds[:, 0] = mins + p * diffs
    # bounds[:, 1] = maxs - p * diffs
    # return bounds

In [34]:
bounds = get_bounds_for_each_feature(pure_sample_without_debris, 1)
bounds

array([[ 3.70790000e+05,  1.56484208e+08],
       [ 1.25800000e+03,  5.28370000e+04],
       [ 1.83000000e+02,  6.22300000e+03],
       [ 3.50716500e+06,  3.08605280e+08],
       [ 1.25120000e+04,  5.37830000e+04],
       [ 2.77000000e+02,  7.01000000e+03],
       [-4.27441000e+05,  2.98725340e+07],
       [ 9.20000000e+01,  1.16880000e+04],
       [-1.21100000e+03,  5.61400000e+03],
       [ 2.69894000e+05,  6.83751520e+07],
       [ 1.51700000e+03,  1.84730000e+04],
       [ 1.65000000e+02,  7.01700000e+03],
       [-3.61384080e+07,  2.55806352e+08],
       [ 1.42100000e+04,  5.55080000e+04],
       [-2.23200000e+03,  6.38500000e+03],
       [ 0.00000000e+00,  3.07000000e+02],
       [ 1.00000000e+00,  1.57700000e+03],
       [ 2.30000000e+01,  1.48300000e+03],
       [ 0.00000000e+00,  1.68000000e+02],
       [ 0.00000000e+00,  1.49000000e+02],
       [ 0.00000000e+00,  3.73000000e+02],
       [ 3.84000000e+02,  1.06590000e+04],
       [ 8.20000000e+02,  5.56660000e+04],
       [ 7.

In [35]:
# mins = pure_sample_without_debris.min(axis=0)
# maxs = pure_sample_without_debris.max(axis=0)
# print(get_bounds_for_each_feature(pure_sample_without_debris, 1))
# (np.stack((mins, maxs), axis=1) == get_bounds_for_each_feature(pure_sample_without_debris, 1)).all()

In [36]:
# get_bounds_for_each_feature(pure_sample_without_debris, 0)
# print(get_bounds_for_each_feature(pure_sample_without_debris, 0))
# (np.median(pure_sample_without_debris, axis=0) == get_bounds_for_each_feature(pure_sample_without_debris, 0)[:, 0]).all()

In [37]:
np.isnan(bounds).any()

False

# Apply model to test set
implement it myself because not sure if there is a better way

In [38]:
np.isnan(test_data).any()

False

In [39]:
test_data.shape

(63928, 448)

In [40]:
test_data.shape

(63928, 448)

In [41]:
def predict(test_data, bounds):
    # 1 means inside the gate, 0 means outside
    return ((test_data >= bounds[:, 0]) & (test_data <= bounds[:, 1])).all(axis=1).astype(int)

In [42]:
predictions = predict(test_data[:, :-1], bounds)
predictions.shape

(63928,)

In [43]:
predictions.sum()

426

# Quantify the quality of the prediction

In [44]:
# a 1 in the predition just means this event is thought to be isnide the gate
# (meaning being a hit), but here I need to know which sample (=label) I tried to predict
def quantify_results(predictions, sample_labels, tiff_labels):
    output = np.zeros((2, 4))
    index = 0
    for class_label in [0, 1]:
        print(f'class {class_label}')
        for tiff_label in [0, 1]:
            print(f'tiff {tiff_label}')
            mask = (sample_labels == class_label) & (tiff_labels == tiff_label)
            num_samples_in_this_category = mask.sum()
            hits_in_this_category = predictions[mask].sum()
            output[:, index] = [hits_in_this_category, num_samples_in_this_category-hits_in_this_category]
            index += 1
    return output

In [45]:
res = quantify_results(predictions, test_labels, test_data[:, -1])
res

class 0
tiff 0
tiff 1
class 1
tiff 0
tiff 1


array([[0.0000e+00, 4.2600e+02, 0.0000e+00, 0.0000e+00],
       [4.4392e+04, 1.4000e+01, 1.7578e+04, 1.5180e+03]])

row 1: the hits I have (events that are predicted to be class 0 in this case and having a tiff)

row 2: are all the others

-----

column 1: class 0 and no tiff

column 2: class 0 and tiff

column 3: class 1 and no tiff

column 4: class 1 and tiff

basically full purity, 426 hits, no false positives, missing 14

# Repeat for the other class

In [46]:
pure_sample_without_debris = filter_pure_sample_without_debris(train_data, train_labels, label_to_use=1)
bounds = get_bounds_for_each_feature(pure_sample_without_debris, 1)
predictions = predict(test_data[:, :-1], bounds)
res = quantify_results(predictions, test_labels, test_data[:, -1])
res

class 0
tiff 0
tiff 1
class 1
tiff 0
tiff 1


array([[0.0000e+00, 0.0000e+00, 0.0000e+00, 1.4870e+03],
       [4.4392e+04, 4.4000e+02, 1.7578e+04, 3.1000e+01]])

similar results, full purity, 1487 hits, no false positives, missing 31

# Summary

Both were run using full min max range (p=1). Josh ran the same experiment independently and had the same results. Right now I am using all features (no ranking). This works surprisingly weel. Should be tested if this is still the case for a more complicated test sample (more species mixed). If the purity/precision is worse, one can try making `p` smaller and accepting to miss more positives. If using all features is not possible, one need to find a way of ranking and choosing the most important features.